## Create FHIR R4 CapStatement Resource


### Outline:

- Source excel with requirements
- pandas to convert in python Ordered Dict
- build json
- generate narrative using Jinja2 templates

### Prerequisites:

- Python 3.6 or greater

## STEP1: Choose Source Spreadsheet to use

*note:  Jupyteralab and widgets issues: see https://stackoverflow.com/questions/4
9542417/how-to-get-ipywidgets-working-in-jupyter-lab for solution 

In [14]:
#******************** NEED TO UPDATE WHEN Adding new IGS ************************************************

from os import name as os_name
my_base = '/Users/ehaas/' if os_name == 'posix' else '//ERICS-AIR-2/ehaas/'


in_path_list = [
        "---pick one below---",
        "Documents/Python/MyNotebooks/CapStatement/temp_source_spreadsheets/test-spreadsheet.xlsx",
        "Documents/FHIR/US-Core-R4/input/resources_spreadsheets/uscore-client.xlsx",
        "Documents/FHIR/US-Core-R4/input/resources_spreadsheets/uscore-server.xlsx",
        'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-initiator.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/notification-forwarder.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-receiver.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-responder.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-requester.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Reporter_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Server.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Server.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Receiver_Server.xlsx',
         "Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Reporter_Client.xlsx",
          "Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Receiver_Server.xlsx",
         'C:/Users/Administrator/Downloads/plan-net-server.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-source-client.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-source-server.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-consumer-client.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-consumer-server.xlsx',
        'Documents/FHIR/Argo-PL/input/resources-spreadsheet/client.xlsx',
        'Documents/FHIR/Argo-PL/input/resources-spreadsheet/server.xlsx',   
        ]


# ----------spreadsheet source---------------
from IPython.display import display as Display, HTML, Markdown, Javascript
from ipywidgets import Dropdown
menu = Dropdown(
       options=[my_base + x for x in in_path_list],
       description='Choose Spreadsheet Source file',
       style = {'description_width': 'initial',},
       layout={'width': 'initial'},
        )


menu

Dropdown(description='Choose Spreadsheet Source file', layout=Layout(width='initial'), options=('/Users/ehaas/…

## STEP2:  *CLICK HERE* and then 'Select Run Selected Cell and All Below'  from menu bar to continue"

In [131]:
xls = menu.value
xls

'/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-source-client.xlsx'

### Import FHIRClient and other libraries

In [132]:
%config IPCompleter.greedy=True

In [133]:
from fhir.resources import construct_fhir_element
from json import dumps, loads
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date, timezone, timedelta
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from commonmark import commonmark
from htmlmin import minify

####  Assign Global Variables

Here is where we assign all the global variables for this example such as the canonical base and project information

In [134]:
fhir_base_url = 'http://hl7.org/fhir/'
f_jurisdiction =  construct_fhir_element('CodeableConcept',{
      "coding" : [
        {
          "system" : "urn:iso:std:iso:3166",
          "code" : "US"
        }
      ]
    })

conf_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation'
combo_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-search-parameter-combination'
sp_specials = {'us-core-includeprovenance':'http://hl7.org/fhir/us/core/SearchParameter/us-core-includeprovenance'}  # dict to for SP to get right canonicals, may use spreadsheet or package file in future.

none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', None,]

sep_list = (',', ';', ' ', ', ', '; ')


timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
f_now = datetime.now(tzinfo)
f_now

datetime.datetime(2022, 2, 28, 19, 34, 33, 446501, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600)))

#### To PascalCase Utility

In [135]:
def kebab_to_pascal(word):
    return ''.join(x.capitalize() for x in word.split('-'))

#### Set Simple Attributes

In [136]:
def set_attr(v=None):
    if v and v not in none_list:
        return v
    else:
        return

#### Set List Attributes

In [137]:
def set_attr_list(v=None):
    if v:
        return [i for i in v.split(",")]
    else:
        return

#### Conformance Extension

In [138]:
def get_conf(conf='MAY',as_dict=False):
    if as_dict:
        return [construct_fhir_element('Extension',dict(
            url = conf_url,
            valueCode = conf
            )).json()]
    else:
        return [construct_fhir_element('Extension',dict(
            url = conf_url,
            valueCode = conf if conf else "MAY"
            ))]
        

#### Primitive Conformance Extension

In [139]:
def get_prim_ext(conf=None):
    if not set(conf).issubset(none_list):
        conf_list = []
        try: # is comma sep string
            conf = conf.split(',')
        except AttributeError: # is list
            pass         
        for i in conf:
            if i not in none_list:
                conf_ext = construct_fhir_element('FHIRPrimitiveExtension', dict(
                    extension = get_conf(conf=i),
                    ))
            else:
                conf_ext = None
            conf_list.append(conf_ext)
        
        return conf_list
    else:
        return

### Addin Extensions

In [140]:
def get_addin_ext(py_ext, json_ext):            
    print(py_ext)
    if json_ext:   # ie not ''
        addin_ext = construct_fhir_element('Extension',loads(json_ext))            
        # addin_ext.extension =  get_conf('SHALL') violates invariant   - DONT USE           
        print(addin_ext)                 
        py_ext.append(addin_ext) # add in other extensions
    print(py_ext)
    return py_ext

### validate

In [141]:
# *********************** validate Resource ********************************

def validate(r):

    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/CapabilityStatement/$validate', params = params, headers = headers, data = r.json())
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Cap Statement input data

function to convert to dataframe series to namedtuple for easy peasy dot notation use.

#### first the config data

In [142]:
df = read_excel(xls,'config',na_filter = False,index_col=0)  # use the index_col = 0 for setting the first rwo as the index

df

,Value
Name,
source,/users/ehaas/Documents/FHIR/Davinci-CDEX/input/
packagepath,http://build.fhir.org/ig/HL7/davinci-ecdx/pack...
pre,Da Vinci
canon,http://hl7.org/fhir/us/davinci-cdex/
publisher,HL7 International - Patient Care Work Group
publishersystem,url
publishervalue,http://www.hl7.org/Special/committees/patientc...
suppress_may_sps,False


#### assign globals e.g. publisher parameter etc...

In [143]:
#df.[Column].[row] to get a value df.loc[[row],[Column]] or df.at[[column],[col]] works too
df.Value.source #, df.loc['source','Value'], df.at['source' ,'Value']

'/users/ehaas/Documents/FHIR/Davinci-CDEX/input/'

In [144]:

ig_source_path = df.Value.source
ig_package_tar_path =  df.Value.packagepath
# --------- ig specific variable -------------------
pre = df.Value.pre  # for Titles - not sure this is actually used
canon = df.Value.canon # don't forget the slash  - fix using os.join or path
#
publisher = df.Value.publisher
#
publisher_endpoint = dict(
                    system = df.Value.publishersystem,
                    value = df.Value.publishervalue,
                  )

pprint(publisher_endpoint)
ig_source_path

{'system': 'url',
 'value': 'http://www.hl7.org/Special/committees/patientcare/index.cfm'}


'/users/ehaas/Documents/FHIR/Davinci-CDEX/input/'

#### Get the meta sheet

In [145]:
df = read_excel(xls,'meta',na_filter = False)

df

,Element,Value
0,id,data-source-client
1,version,1.0.0
2,fhirVersion,4.0.1
3,description,This CapabilityStatement describes the expecte...
4,mode,client
5,documentation,The Da Vinci CDex Data Source **SHALL**:\n\n1...
6,security,1. For general security consideration refer to...
7,format,"json,xml"
8,format_conf,"SHALL,SHOULD"
9,patchFormat,


#### Create NamedTuple from df to use dot notation

In [146]:
d = dict(zip(df.Element, df.Value))
meta = namedtuple("Meta", d.keys())(*d.values())      
         
meta.id

'data-source-client'


### Create CS instance

In [147]:
cs = construct_fhir_element('CapabilityStatement',dict(
id = meta.id,
url = f'{canon}CapabilityStatement/{meta.id}',
version = meta.version,
name = f'{kebab_to_pascal(meta.id)}CapabilityStatement',
title = f'{titlecase(meta.id).replace("Us ", "US ")} CapabilityStatement',
status = 'active',
experimental = False,
date = f_now, # as FHIRDate
publisher = publisher,
contact = [construct_fhir_element('ContactDetail', {"telecom" : [ publisher_endpoint ] })],
description = meta.description,
jurisdiction = [f_jurisdiction],
kind = 'requirements',
fhirVersion = meta.fhirVersion,
format = set_attr_list(meta.format),
format__ext = get_prim_ext(meta.format_conf),
patchFormat = set_attr_list(meta.patchFormat),
patchFormat__ext = get_prim_ext(meta.patchFormat_conf),
))

print(cs.json(indent=2))

{
  "resourceType": "CapabilityStatement",
  "id": "data-source-client",
  "url": "http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-source-client",
  "version": "1.0.0",
  "name": "DataSourceClientCapabilityStatement",
  "title": "Data Source Client CapabilityStatement",
  "status": "active",
  "experimental": false,
  "date": "2022-02-28T19:34:33.446501-08:00",
  "publisher": "HL7 International - Patient Care Work Group",
  "contact": [
    {
      "telecom": [
        {
          "system": "url",
          "value": "http://www.hl7.org/Special/committees/patientcare/index.cfm"
        }
      ]
    }
  ],
  "description": "This CapabilityStatement describes the expected capabilities of a Da Vinci CDex Data Source (often an EHR) in *Client* mode during a clinical data exchange with a Data Consumer. This includes the following interactions:\n1. POSTing the `$submit-attachment` operation to exchange clinical data using a FHIR based Attachments transaction.\n1. Query for Autho

### Add in Rest

In [148]:
def get_sys_op():
    op_list = []
    df_op = read_excel(xls,'ops',na_filter = False)
    for i in df_op.itertuples(index=True):
        if i.type == 'system':
            op = construct_fhir_element('CapabilityStatementRestResourceOperation', dict(
            #op = CS.CapabilityStatementRestResourceOperation()
            name = i.name,
            definition = i.definition,
            documentation = i.documentation if i.documentation not in none_list else None,
            extension = get_conf(i.conf), 
            ))
            op_list.append(op.json())
    return op_list if op_list else None


def get_rest_ints():
    ri_list = []
    df_ri = read_excel(xls,'rest_interactions',na_filter = False)
    for i in df_ri.itertuples(index=True):
        ri = construct_fhir_element('CapabilityStatementRestInteraction', dict(
        code = i.code,
        documentation = i.doc if i.doc not in none_list else None,
        extension = get_conf(i.conf),
        ))                         
        print(ri.json(indent=2))
        ri_list.append(ri.json())        
    return ri_list  if ri_list else None

rest = construct_fhir_element('CapabilityStatementRest',(dict(
    mode = meta.mode,
    documentation = meta.documentation,
    security = dict(
        description = meta.security
        ) if meta.security else None,
    interaction = get_rest_ints(),
    operation = get_sys_op()
    )))
                              
cs.rest = [rest]

print(cs.json(indent=2))

{
  "extension": [
    {
      "url": "http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation",
      "valueCode": "MAY"
    }
  ],
  "code": "transaction"
}
{
  "extension": [
    {
      "url": "http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation",
      "valueCode": "MAY"
    }
  ],
  "code": "batch"
}
{
  "extension": [
    {
      "url": "http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation",
      "valueCode": "MAY"
    }
  ],
  "code": "search-system"
}
{
  "extension": [
    {
      "url": "http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation",
      "valueCode": "MAY"
    }
  ],
  "code": "history-system"
}
{
  "resourceType": "CapabilityStatement",
  "id": "data-source-client",
  "url": "http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-source-client",
  "version": "1.0.0",
  "name": "DataSourceClientCapabilityStatement",
  "title": "Data Source Client CapabilityStatement",
  "status": "acti

### Add in IG, and Capstatements and their conformance

In [149]:
df_igs = read_excel(xls,'igs',na_filter = False)
cs.implementationGuide = set_attr([ig.canonical for ig in df_igs.itertuples(index=True)])
#print([ig.conformance for ig in df_igs.itertuples(index=True) if ig.imports not in none_list])
cs.implementationGuide__ext = get_prim_ext([ig.conformance for ig in df_igs.itertuples(index=True)])

df_capstatements = read_excel(xls,'capstatements',na_filter = False)
cs.imports = set_attr([ig.canonical for ig in df_capstatements.itertuples(index=True) if ig.imports not in none_list])
cs.imports__ext = get_prim_ext([ig.conformance for ig in df_capstatements.itertuples(index=True) if ig.imports not in none_list])
cs.instantiates = set_attr([ig.canonical for ig in df_capstatements.itertuples(index=True) if ig.instantiates not in none_list])
cs.instantiates__ext = get_prim_ext([ig.conformance for ig in df_capstatements.itertuples(index=True) if ig.instantiates not in none_list])

print(cs.json(indent=2))

{
  "resourceType": "CapabilityStatement",
  "id": "data-source-client",
  "url": "http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-source-client",
  "version": "1.0.0",
  "name": "DataSourceClientCapabilityStatement",
  "title": "Data Source Client CapabilityStatement",
  "status": "active",
  "experimental": false,
  "date": "2022-02-28T19:34:33.446501-08:00",
  "publisher": "HL7 International - Patient Care Work Group",
  "contact": [
    {
      "telecom": [
        {
          "system": "url",
          "value": "http://www.hl7.org/Special/committees/patientcare/index.cfm"
        }
      ]
    }
  ],
  "description": "This CapabilityStatement describes the expected capabilities of a Da Vinci CDex Data Source (often an EHR) in *Client* mode during a clinical data exchange with a Data Consumer. This includes the following interactions:\n1. POSTing the `$submit-attachment` operation to exchange clinical data using a FHIR based Attachments transaction.\n1. Query for Autho

### add Resources

- read sheets for resource attributes, interaction attributes,  search attributes, profiles, and combo search parameters

In [150]:
df_resources = read_excel(xls,'resources',na_filter = False)
df_resources = df_resources[df_resources.type.str[0] != '!']
df_resources

,type,conformance,documentation,profile,profile_conf,versioning,versioning_conf,readHistory,readHistory_conf,updateCreate,...,conditionalUpdate,conditionalUpdate_conf,conditionalDelete,conditionalDelete_conf,referencePolicy,referencePolicy_conf,include_conf,include,revinclude,revinclude_conf
0,CommunicationRequest,SHOULD,Required resource type to carry authorization ...,,,,,,,,...,,,,,,,,,,
1,ServiceRequest,SHOULD,Required resource type to carry authorization ...,,,,,,,,...,,,,,,,,,,
2,Subscription,SHOULD,Required resource type to subscribe to data so...,,,,,,,,...,,,,,,,,,,
3,DocumentReference,SHOULD,Required resource type to support the `$submit...,,,,,,,,...,,,,,,,,,,


In [151]:
df_profiles = read_excel(xls,'profiles',na_filter = False)  #df1 = df[df.Hostname.str[0] != "abc"]
df_profiles = df_profiles[df_profiles.Profile.str[0] != '!']
df_profiles

,Profile,Name,url,Conformance,Type
0,http://hl7.org/fhir/us/core/StructureDefinitio...,US Core DocumentReference Profile,http://hl7.org/fhir/us/core/StructureDefinitio...,SHALL,DocumentReference


In [152]:
df_i = read_excel(xls,'interactions',na_filter = False)
df_sp = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)
df_op = read_excel(xls,'ops',na_filter = False)


def get_i(type):
    int_list = []
    for i in df_i.itertuples(index=True):
        #print(i.code, getattr(i,f'conf_{type}'))
        if getattr(i,f'conf_{type}') not in none_list:
            try:
                documentation = getattr(i,f'doc_{type}') if getattr(i,f'doc_{type}') not in none_list else None
            except AttributeError:
                documentation = None
            int  = construct_fhir_element('CapabilityStatementRestResourceInteraction', dict(
            code = i.code,
            documentation = documentation,
            extension = get_conf(getattr(i,f'conf_{type}')),
            ))
            int_list.append(int.json()),               
    return int_list if int_list else None


def get_sp(r_type):
    sp_list = []
    for i in df_sp.itertuples(index=True):
        if i.base == r_type:
            # TODO need to fix this to reference the package file to reconcile definition to names
            if i.code in sp_specials: #special case temp fix for us-core
                definition = sp_specials[i.code]
            elif i.update == 'Y' or i.exists =='N':
                definition = f'{canon}SearchParameter/{pre.lower()}-{i.base.lower()}-{i.code.split("_")[-1]}'                  
            elif i.code.startswith('_'): #common sp
                definition = f'{fhir_base_url}SearchParameter/{i.code.replace("_","Resource-")}'
            else:  # use base definition
                definition = f'{fhir_base_url}SearchParameter/{i.base}-{i.code}'
            # print(definition)
            
            sp  = construct_fhir_element('CapabilityStatementRestResourceSearchParam', dict(
            name = i.code,
            definition = definition,
            documentation = i.documentation if i.documentation not in none_list else None,               
            type = i.type,
            extension = get_conf(i.base_conf),
            ))
            #print(sp.json())                
            sp_list.append(sp.json())                            
    return sp_list if sp_list else None 


def get_combo_ext(r_type,combos):
    x_list = []
    for combo in combos:
        # convert to extension
        combo_ext = construct_fhir_element('Extension', dict (
        url = combo_url,
        extension=get_conf(combo[1]),
        ))
        for param in combo[0].split(','):
            req_combo = construct_fhir_element('Extension', dict (
                    url = 'required',
                    valueString = param,   #http://hl7.org/fhir/us/core/SearchParameter/us-core-patient-family
                ))
            combo_ext.extension.append(req_combo)
        x_list.append(combo_ext)
        # print(x_list)
    return x_list
    
def get_op(r_type):
    op_list = []
    for i in df_op.itertuples(index=True):
         if i.type == r_type:
            op = construct_fhir_element('CapabilityStatementRestResourceOperation', dict(
            name = i.name, 
            definition = i.definition,
            documentation = i.documentation if i.documentation not in none_list else None,
            extension = get_conf(i.conf),
            ))
            try:                     
                op.extension =  get_addin_ext(op.extension, i.ext)
            except AttributeError:
                print("---- no addin extensions found-----")
            op_list.append(op.json())                           
    return op_list if op_list else None 


rest.resource =  []
for r in df_resources.itertuples(index=True):
    # print(r.type, r.conformance, r.readHistory
    supported_profile = [p.Profile for p in df_profiles.itertuples(index=True) if p.Type == r.type]
    supported_profile_ext = get_prim_ext([p.Conformance for p in df_profiles.itertuples(index=True) if p.Type == r.type])
    #pprint(supported_profile)                         
    res = construct_fhir_element('CapabilityStatementRestResource', dict(
        type = r.type,
        documentation = set_attr(r.documentation),
        versioning = set_attr(r.versioning),
        readHistory = set_attr(r.readHistory),
        updateCreate = set_attr(r.updateCreate),
        conditionalCreate = set_attr(r.conditionalCreate),
        conditionalRead = set_attr(r.conditionalRead),
        conditionalUpdate = set_attr(r.conditionalUpdate),
        conditionalDelete = set_attr(r.conditionalDelete),
        referencePolicy = set_attr_list(r.referencePolicy),
        searchInclude = set_attr_list(r.include),
        searchInclude__ext = get_prim_ext(r.include_conf),
        searchRevInclude = set_attr_list(r.revinclude),
        searchRevInclude__ext = get_prim_ext(r.revinclude_conf),
        interaction = get_i(r.type),
        searchParam = get_sp(r.type),
        operation = get_op(r.type),
        profile = set_attr(r.profile),
        supportedProfile = supported_profile if supported_profile else None,
        supportedProfile__ext = supported_profile_ext
        )
    )
    res.extension = get_conf(r.conformance)
    combos = {(i.combo,i.combo_conf) for i in df_combos.itertuples(index=True) if i.base == r.type}
    res.extension = res.extension + get_combo_ext(r.type,combos) # convert list to  lst of combo extension                              
    rest.resource.append(res)

rest.resource =  sorted(rest.resource,key = lambda x: x.type)  # sort resources                         
cs.rest = [rest]
    
print(cs.json(indent=2))          

{
  "resourceType": "CapabilityStatement",
  "id": "data-source-client",
  "url": "http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-source-client",
  "version": "1.0.0",
  "name": "DataSourceClientCapabilityStatement",
  "title": "Data Source Client CapabilityStatement",
  "status": "active",
  "experimental": false,
  "date": "2022-02-28T19:34:33.446501-08:00",
  "publisher": "HL7 International - Patient Care Work Group",
  "contact": [
    {
      "telecom": [
        {
          "system": "url",
          "value": "http://www.hl7.org/Special/committees/patientcare/index.cfm"
        }
      ]
    }
  ],
  "description": "This CapabilityStatement describes the expected capabilities of a Da Vinci CDex Data Source (often an EHR) in *Client* mode during a clinical data exchange with a Data Consumer. This includes the following interactions:\n1. POSTing the `$submit-attachment` operation to exchange clinical data using a FHIR based Attachments transaction.\n1. Query for Autho

### Validate

In [153]:
 #validate and write to file

print('...validating')
r = validate(cs)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

...validating


### Create Narrative

- Using Jinja2 Template create xhtml for narrative

#### First: Get spec_internal from package.tgz a json file which includes canonical to local relative page links

Note for this to work you have to have a working build that already contains all the needed artifacts.

In [154]:
import tarfile
package_path = Path.cwd() / 'tarfiles'/'package.tgz'  #get_si(path)
print(package_path)
def get_si(package_path):
    with tarfile.open(package_path, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return (loads(r))

    
def get_si3(path):
    tf = get(path)
    if tf.status_code == 200:
       print(f'get tarfile from {ig_package_tar_path}')
    return tf

"e.g. https://build.fhir.org/ig/HL7/davinci-deqm/package.tgz" 
try:   
    tf= get_si3(ig_package_tar_path) # get from remote server
except:
   in_path = Path() / package_path
   print(in_path)
   tf = in_path.read_bytes()
   package_path.write_bytes(tf)  # get from package (json) file in local .fhir directory
else:
    package_path.write_bytes(tf.content)    #save in temp file
    
si = get_si(package_path) #unpack from file

path_map = si['paths']
path_map

/Users/ehaas/Documents/Python/MyNotebooks/CapStatement/tarfiles/package.tgz
get tarfile from http://build.fhir.org/ig/HL7/davinci-ecdx/package.tgz


{'http://hl7.org/fhir/us/davinci-cdex/ImplementationGuide/hl7.fhir.us.davinci-cdex|1.0.0': '1.0.0/ImplementationGuide-hl7.fhir.us.davinci-cdex.html',
 'http://hl7.org/fhir/us/davinci-cdex/ImplementationGuide/hl7.fhir.us.davinci-cdex': 'ImplementationGuide-hl7.fhir.us.davinci-cdex.html',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request|1.0.0': '1.0.0/StructureDefinition-cdex-task-data-request.html',
 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request': 'StructureDefinition-cdex-task-data-request.html',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-server|1.0.0': '1.0.0/CapabilityStatement-data-consumer-server.html',
 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-server': 'CapabilityStatement-data-consumer-server.html',
 'http://hl7.org/fhir/us/davinci-cdex/ValueSet/cdex-attachment-reason|1.0.0': '1.0.0/ValueSet-cdex-attachment-reason.html',
 'http://hl7.org/fhir/us/davinci-cde

#### Then Use Jinja2 template to create narrative

In [155]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

print(cs.date)

2022-02-28 19:34:33.446501-08:00


In [156]:
def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'],),
    trim_blocks = True,
    lstrip_blocks = True,
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}
purl_map = {p.Profile:p.url if p.url not in none_list else p.Profile for p in df_profiles.itertuples(index=True)}
igname_map = {ig.canonical:ig.name for ig in df_igs.itertuples(index=True)}
igurl_map = {ig.canonical:ig.url if ig.url not in none_list else ig.canonical for ig in df_igs.itertuples(index=True)}
csname_map = {cs.canonical:cs.name for cs in df_capstatements.itertuples(index=True)}
csurl_map = {cs.canonical:cs.url if cs.url not in none_list else cs.canonical for cs in df_capstatements.itertuples(index=True)}
print(csurl_map)
print([i for i in df_capstatements.itertuples(index=True)])
rendered = template.render(cs=cs, path_map=path_map, pname_map=pname_map, purl_map=purl_map, sp_map=sp_map, 
                           csname_map=csname_map, csurl_map=csurl_map, igname_map=igname_map, igurl_map=igurl_map)


{'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-client': 'http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-client', 'http://hl7.org/fhir/us/core/CapabilityStatement/us-core-client': 'http://hl7.org/fhir/us/core/CapabilityStatement-us-core-client.html'}
[Pandas(Index=0, imports='', instantiates=True, name='Data Consumer Client CapabilityStatement', canonical='http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-client', url='http://hl7.org/fhir/us/davinci-cdex/CapabilityStatement/data-consumer-client', conformance='SHALL'), Pandas(Index=1, imports='', instantiates=True, name='US Core Client CapabilityStatement', canonical='http://hl7.org/fhir/us/core/CapabilityStatement/us-core-client', url='http://hl7.org/fhir/us/core/CapabilityStatement-us-core-client.html', conformance='SHALL')]


### Minify the xhtml

In [157]:
def x_minify(xhtml):
    h_min=minify(xhtml, remove_optional_attribute_quotes=False, remove_comments=True)
    x_min = h_min.replace('<br>','<br />')
    x_min = x_min.replace('<hr>','<hr />')
    return x_min

mini = x_minify(rendered)
#print(type(mini))
#display(HTML(rendered))
display(HTML(mini))

'''
#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative_pre.xhtml'
path.write_text(rendered, encoding="utf-8")
path = Path.cwd() / 'debug' / 'narrative_mini_pre.xhtml'
path.write_text(mini, encoding="utf-8")
#===================================================
'''
narr = construct_fhir_element('Narrative', dict(
        status = 'generated',
        div = mini,
        ))
cs.text = narr
#print(cs.json(indent=2))

Resource Type,Supported Profiles,Supported Searches,Supported _includes,Supported _revincludes,Supported Operations
CommunicationRequest,-,-,-,-,-
DocumentReference,US Core DocumentReference Profile,-,-,-,-
ServiceRequest,-,-,-,-,-
Subscription,-,-,-,-,-


### validate again

In [158]:
print('...validating')
r = validate(cs)
d = display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
           
#======== write to temp file to debug =======
from html.parser import HTMLParser

class HTMLFilter(HTMLParser):
    text = ""
    def handle_data(self, data):
        self.text += data

f = HTMLFilter()
f.feed(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}')
path = Path.cwd() / 'debug' / 'validation.txt'
path.write_text(f.text)
#===================================================


...validating


6410

### Write to folder

In [159]:
# save to file
#save in ig_source folder
#ig_source_path = ''

path = Path.cwd()/ my_base / ig_source_path / 'resources' / f'capabilitystatement-{cs.id.lower()}.json'

#path = Path.cwd() /  'resources' / f'capabilitystatement-{cs.id.lower()}.json' # write locally 


print(f'...........saving to file {path}............')
path.write_text(cs.json(indent=2))

...........saving to file /users/ehaas/Documents/FHIR/Davinci-CDEX/input/resources/capabilitystatement-data-source-client.json............


25707